# Exosystems

This program evaluates the deflection of light and the direction error of the Earth from some exosystems.

## Libraries

In [22]:
import numpy as np
import numpy.linalg as LA
import pandas as pd
from astronavigation.planets import SolarSystem, Body
from astropy import constants
from astronavigation.deflection import *
from astronavigation.read_exo import *
from astronavigation.save_df import save_df

## Useful objects

In [3]:
# Define constants
pc = constants.pc.to('km').value
AU = constants.au.to('km').value
c = constants.c.to('km/s').value
eps = 1/c

# create solar system and jupiter
ss = SolarSystem()

v_null = np.array([0, 0, 0])

## Catalogue

In [4]:
path = 'exo_archive.csv'
catalogue = pd.read_csv(path)
catalogue.head()

,pl_name,hostname,pl_orbsmax,pl_radj,pl_massj,st_rad,st_mass,ra,dec,sy_dist,sy_gaiamag
0,Proxima Cen b,Proxima Cen,0.0485,NaN,NaN,0.14,0.12,217.393466,-62.676182,1.30119,8.95361
1,eps Eri b,eps Eri,3.5000,NaN,NaN,0.76,0.81,53.228431,-9.458172,3.20260,3.47719
2,GJ 887 b,GJ 887,0.0680,NaN,NaN,0.47,0.49,346.466827,-35.853069,3.29000,NaN
3,GJ 887 c,GJ 887,0.1200,NaN,NaN,0.47,0.49,346.466827,-35.853069,3.29000,NaN
4,Ross 128 b,Ross 128,0.0496,NaN,NaN,0.20,0.17,176.937604,0.799290,3.37454,9.59067


## Choose systems 

In [33]:
systems = ['K2-285', 'TOI-561', 'TOI-178']

# loop over system
for system_name in systems:
    
    # read all bodies of the system
    system = getExoSystem(system_name, catalogue)
    
    # create a dict of bodies with the desired configuration
    bodies = {}
    
    # read mean orbital radius and create body
    for body_name in system:
        
        # discriminate the star from the planets
        if body_name != system_name:
            
            filt = catalogue['pl_name'] == body_name
            df_row = catalogue.loc[filt]
            
            orb_rad = df_row['pl_orbsmax'].values[0]  # estract value
        else:
            orb_rad = 0
            
        body = system[body_name]
        body.set_Pos( orb_rad*AU*np.array([0, 1, 0]) )
        
        bodies[body_name] = body
        
    # define observer and target
    x_obs = AU*np.array([0, -1, 0])
    
    filt = catalogue['hostname'] == system_name
    df_row = catalogue.loc[filt]
    dist = df_row['sy_dist'].values[0]
    
    x_ref = dist*pc*np.array([0, 1, 0])
    
    dl = []
    # loop over system bodies
    for name, pl in bodies.items():
        
        # check
        if pl.radius==0 or pl.mass==0:
            print(f'ATTENTION!! Planet {name} does not have in the catalgue the value of mass or radius')
        
        # grazing impact angle
        chi = pl.radius/LA.norm(x_obs - pl.pos)
        
        # direction
        l0 = -np.array([np.sin(chi), np.cos(chi), 0])
        x = -np.linalg.norm(x_ref - x_obs) * l0 + x_obs
        
        # deflection
        dls = light_defl(l0, x, pl.pos, x_obs, eps, pl.mass, method='RAMOD')
        dl.append(LA.norm(dls))
        
        print(f'----------------------\n'
              f'body: {name}\n'
              f'defl: {rad2muas(dl[-1])} muas\n'
              f'dir_error: {dist*pc*dl[-1]/AU} AU')
        
    dl = np.array(dl)
    
    dr = dist*pc*dl/AU
    
    # save data
    rows = bodies.keys()
    columns = ['defl[muas]', 'dr[AU]']
    data = [np.round(rad2muas(dl), 2),
            np.round(dr, 4)]
    path = f'Data/exo_defl_{system_name}_latex'
        
    save_df(data, columns, rows, path)

----------------------
body: K2-285 b
defl: 2193.9993814995732 muas
dir_error: 0.3399843381565553 AU
----------------------
body: K2-285 c
defl: 2605.668382373966 muas
dir_error: 0.40377697820105213 AU
----------------------
body: K2-285 d
defl: 1539.641541531116 muas
dir_error: 0.23858439291720324 AU
----------------------
body: K2-285 e
defl: 3219.6780731538292 muas
dir_error: 0.4989245338939905 AU
----------------------
body: K2-285
defl: 1840385.9723386266 muas
dir_error: 285.1880506595659 AU
----------------------
body: TOI-561 b
defl: 655.0657081195487 muas
dir_error: 0.05620398269094916 AU
----------------------
body: TOI-561 c
defl: 1414.6080729997343 muas
dir_error: 0.1213719580553042 AU
----------------------
body: TOI-561 d
defl: 2768.2033350589522 muas
dir_error: 0.23750907794472303 AU
----------------------
body: TOI-561 e
defl: 3519.288221056051 muas
dir_error: 0.3019514100783881 AU
----------------------
body: TOI-561 f
defl: 758.7876096472346 muas
dir_error: 0.065103218